### Step 1. Obtain raw data from Graphite database

In [2]:
%run include/required_libs.py
%matplotlib inline

In [3]:
import requests

In [4]:
partitions = {
    'sandyb' : '20130512',
    'broadwl' : '20160910'
}

for part in partitions:

    prefix = "http://graphite.rcc.uchicago.edu/render/?format=csv" + \
        "&from=" + partitions[part] + \
        "&target=summarize(slurm.midway.bypartition." + part + ".cores.*, \"1day\", \"avg\")"
    
    print(prefix)
    
    r = requests.get(prefix)
    csv = r.text.replace("\"summarize(slurm.midway.bypartition." + part + ".cores.", '')
    csv = csv.replace(', \"\"1day\"\", \"\"avg\"\")\"', '')
    csv = csv.replace(' 00:00:00', '')
    
    with open("rawdata/cluster_load_" + part + ".csv", "w") as f:
        f.write("status,date,cores\n")
        f.write(csv)

http://graphite.rcc.uchicago.edu/render/?format=csv&from=20130512&target=summarize(slurm.midway.bypartition.sandyb.cores.*, "1day", "avg")
http://graphite.rcc.uchicago.edu/render/?format=csv&from=20160910&target=summarize(slurm.midway.bypartition.broadwl.cores.*, "1day", "avg")


### Step 2. Check raw data and understand the columns

In [5]:
bro = pd.read_csv('rawdata/cluster_load_broadwl.csv').dropna()
san = pd.read_csv('rawdata/cluster_load_sandyb.csv').dropna()

Column 1:status

Column 2:date

Column 3:cores

### Step 3. Data gathering

We notice that the first column is a categorial variable, and for each category there are same numbers of rows. So we can gather them into new columns. Save the results into "rawdata/cluster_load_sandyb(browdwl)_gathered.csv"

Reference: https://hackernoon.com/reshaping-data-in-python-fa27dda2ff77

In [6]:
bro.pivot(index='date', columns='status', values='cores').\
to_csv("rawdata/cluster_load_broadwl_gathered.csv")

san.pivot(index='date', columns='status', values='cores').\
to_csv("rawdata/cluster_load_sandyb_gathered.csv")

### Step 4. Refine data

**4.1** For each table, choose "date" and "total" columns, add a new column labeling the partition name. Then bind two tables together. Save it as "cluster_total_cores.csv"

In [7]:
bro = pd.read_csv('rawdata/cluster_load_broadwl_gathered.csv')
bro = bro[bro['date']>'2016-09-30']
bro['partition'] = 'broadwl'
san = pd.read_csv('rawdata/cluster_load_sandyb_gathered.csv')
san['partition'] = 'sandyb'
bro.append(san).to_csv('rawdata/cluster_total_cores.csv', index = False)

**4.2** Add a new column of "su" calculated by "total" x 24

In [8]:
allcore = pd.read_csv('rawdata/cluster_total_cores.csv', parse_dates=['date'])
allcore['su'] = allcore['total'] * 24

**4.3** Modify su by multiplying 1.3 for broadwl partition from 04/01/2017 

In [9]:
allcore.loc[(allcore.partition== 'broadwl') & (allcore.date >= '2017-04-01') ,'su'] = allcore['su'] * 1.3
allcore = allcore.sort_values('date')
allcore = allcore.reset_index(drop = True)

**4.4** Create a new column "cycle" and calculate cycle id (2013-2018) by date

In [10]:
allcore['cycle'] = (pd.to_numeric(allcore['date'].dt.strftime('%Y')) + \
    pd.to_numeric(allcore['date'].dt.strftime('%m'))/12 - 10/12 + 1).astype(int)

**4.5** Make subtotal of "days" and "su" grouped by cycle and partition

In [13]:
allyear = allcore.groupby(['cycle', 'partition']).agg({'cycle':'count', 'su': 'sum'})\
    .rename(columns={'cycle':'days'}).reset_index()
allyear

,cycle,partition,days,su
0,2013,sandyb,122,1.014189e+07
1,2014,sandyb,365,3.962881e+07
2,2015,sandyb,365,3.493516e+07
3,2016,sandyb,365,3.499911e+07
4,2017,broadwl,364,8.931714e+07
5,2017,sandyb,364,3.792839e+07
6,2018,broadwl,241,7.446849e+07
7,2018,sandyb,241,2.646682e+07


**4.6** Create new column as "su_projected" to correct results projected in 365 days.

In [15]:
allyear['su_projected'] = allyear['su'] * 365 / allyear['days']
allyear

,cycle,partition,days,su,su_projected
0,2013,sandyb,122,1.014189e+07,3.034253e+07
1,2014,sandyb,365,3.962881e+07,3.962881e+07
2,2015,sandyb,365,3.493516e+07,3.493516e+07
3,2016,sandyb,365,3.499911e+07,3.499911e+07
4,2017,broadwl,364,8.931714e+07,8.956252e+07
5,2017,sandyb,364,3.792839e+07,3.803259e+07
6,2018,broadwl,241,7.446849e+07,1.127842e+08
7,2018,sandyb,241,2.646682e+07,4.008460e+07


**4.7** Aggrate to cycles and save the tabel as "rawdata/cluster_cores_by_cycles.csv"

In [94]:
cluster = allyear.pivot_table(index='cycle', columns='partition', values='su_projected')\
    .reset_index().fillna(0)
cluster['total_su'] = cluster['sandyb'] + cluster['broadwl']
cluster.to_csv('rawdata/cluster_cores_by_cycles.csv', index=False)

In [96]:
cluster['total_su']=cluster['total_su']/1000000
cluster

partition,cycle,broadwl,sandyb,total_su
0,2013,0.000000e+00,2.792511e+07,27.925112
1,2014,0.000000e+00,3.962881e+07,39.628809
2,2015,0.000000e+00,3.493516e+07,34.935164
3,2016,0.000000e+00,3.499911e+07,34.999115
4,2017,8.956252e+07,3.803259e+07,127.595111
5,2018,1.129426e+08,4.008448e+07,153.027033
